# A. Exploratory Data Analysis

Langkah pertama yang saya lakukan adalah import beberapa library yang nantinya membantu saya untuk menjalankan analisa dan model nantinya. Lanjutannya, saya akan melakukan load dataset serta explorasi data mulai dari shape hingga informasi dari dataset tersebut.

In [ ]:
import zipfile
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models
from skimage.metrics import structural_similarity as ssim

In [ ]:
zip_path = "/content/sample_data/A_23-20250627T052734Z-1-001.zip"
folder_path = "/content/sample_data/images/"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(folder_path)

image_files = []
for root, dirs, files in os.walk(folder_path):
    for file in files:
        if file.lower().endswith('.jpg'):
            image_files.append(os.path.join(root, file))

print(f"Jumlah gambar: {len(image_files)}")

Jumlah gambar: 1074


Tahap ini menampilkan gambar-gambar dalam batch berukuran 18, ditata dalam grid dengan 6 kolom per baris. Setiap batch ditampilkan dalam satu halaman (figure) menggunakan `matplotlib`, dengan pengaturan ukuran otomatis berdasarkan jumlah baris yang dibutuhkan. Nama file ditampilkan sebagai judul tiap gambar, dan sumbu dihilangkan agar fokus ke visual. Tujuannya untuk memvisualisasikan sekumpulan gambar secara terstruktur dalam grid, sehingga memudahkan evaluasi kualitas, variasi, atau hasil dari proses seperti pelatihan model atau generasi gambar secara menyeluruh.


In [ ]:
batch_size = 18
cols = 6

for start in range(0, len(image_files), batch_size):
    page = image_files[start:start + batch_size]
    rows = (len(page) + cols - 1) // cols

    plt.figure(figsize=(18, 3.5 * rows))
    for idx, img_file in enumerate(page):
        image = mpimg.imread(img_file)
        plt.subplot(rows, cols, idx + 1)
        plt.imshow(image)
        plt.title(os.path.basename(img_file), fontsize=7)
        plt.axis('off')

    plt.tight_layout()
    plt.show()

Selanjutnya, tahap ini bertujuan untuk memfilter dan menganalisis gambar yang mengandung kata *"kirmizi"* dalam nama filenya. Gambar-gambar tersebut dibuka maksimal 4 buah, diubah ke mode grayscale, lalu dianalisis distribusi intensitas cahayanya.

Setiap gambar divisualisasikan bersama histogram intensitas pixelnya (0–255), dengan garis vertikal merah menunjukkan nilai rata-rata intensitas. Ini membantu memahami karakteristik pencahayaan atau kontras gambar, sekaligus mendeteksi gambar rusak melalui pengecekan. Informasi penting seperti ukuran, mode, dan rata-rata intensitas juga dicetak untuk setiap gambar.


In [ ]:
kirmizi_images = [
    os.path.join(root, file)
    for root, _, files in os.walk(folder_path)
    for file in files
    if file.lower().endswith('.jpg') and 'kirmizi' in file.lower()
]

print(f"Total 'kirmizi' images found: {len(kirmizi_images)}\n")

max_display = 4
image_info = []

for img_path in kirmizi_images[:max_display]:
    try:
        with Image.open(img_path) as temp:
            temp.verify()
        with Image.open(img_path) as img:
            gray = img.convert('L')
            arr = np.array(gray)

            image_info.append({
                'filename': os.path.basename(img_path),
                'gray_img': gray,
                'arr': arr,
                'mean': arr.mean(),
                'size': gray.size,
                'mode': gray.mode
            })

    except Exception as e:
        print(f"[ERROR] {os.path.basename(img_path)} — cannot open ({e})")

for i in range(0, len(image_info), 2):
    subset = image_info[i:i+2]
    fig, axes = plt.subplots(2, 2, figsize=(12, 6))

    for j, info in enumerate(subset):
        axes[0, j].imshow(info['gray_img'], cmap='gray')
        axes[0, j].set_title(f"Grayscale - {info['filename']}", fontsize=9)
        axes[0, j].axis('off')

        axes[1, j].hist(info['arr'].ravel(), bins=64, color='gray', alpha=0.8)
        axes[1, j].set_title('Histogram Intensitas', fontsize=9)
        axes[1, j].set_xlabel('Intensity')
        axes[1, j].set_ylabel('Pixel Count')
        axes[1, j].set_xlim(0, 255)
        axes[1, j].axvline(info['mean'], color='red', linestyle='--', linewidth=1, label='Mean')
        axes[1, j].legend(fontsize=7)
        axes[1, j].grid(True, linestyle='--', alpha=0.3)

        print(f"Filename: {info['filename']}")
        print(f" - Size  : {info['size']}")
        print(f" - Mode  : {info['mode']}")
        print(f" - Mean Intensity : {info['mean']:.1f}")
        print("-" * 40)

    plt.tight_layout()
    plt.show()

Tahap ini bertujuan untuk menampilkan detail dan visualisasi warna dari maksimal 5 gambar *"kirmizi"* secara berpasangan. Untuk setiap gambar, program menampilkan ukuran, mode warna, serta menghitung dan memvisualisasikan rata-rata nilai RGB jika mode-nya adalah RGB. Gambar ditampilkan di atas, dan grafik batang rata-rata warna ditampilkan di bawah. Jika mode bukan RGB, ditampilkan teks pemberitahuan sebagai pengganti grafik warna. Tahap ini membantu memahami karakteristik warna dominan dari gambar secara intuitif.


In [ ]:
for i in range(0, min(5, len(kirmizi_images)), 2):
    fig, axes = plt.subplots(2, 2, figsize=(10, 5))

    for j in range(2):
        idx = i + j
        if idx >= len(kirmizi_images):
            break

        file = kirmizi_images[idx]
        img_path = os.path.join(folder_path, file)
        with Image.open(img_path) as img:
            img_array = np.array(img)

            print(f"{idx+1}. {file} -> Ukuran: {img.size}, Mode: {img.mode}")

            if img.mode == 'RGB':
                avg_rgb = np.mean(img_array.reshape(-1, 3), axis=0)
                print(f"   Rata-rata RGB: R={avg_rgb[0]:.1f}, G={avg_rgb[1]:.1f}, B={avg_rgb[2]:.1f}")
            else:
                avg_rgb = None
                print("   Mode bukan RGB, lewati visualisasi warna.")

            axes[0, j].imshow(img)
            axes[0, j].axis('off')
            axes[0, j].set_title(file)

            if avg_rgb is not None:
                colors = ['r', 'g', 'b']
                axes[1, j].bar(colors, avg_rgb, color=colors)
                axes[1, j].set_ylim(0, 255)
                axes[1, j].set_title('Rata-rata RGB')
                axes[1, j].set_ylabel('Nilai warna')
            else:
                axes[1, j].text(0.5, 0.5, "Mode bukan RGB\ntidak ada data warna",
                                horizontalalignment='center',
                                verticalalignment='center',
                                fontsize=12)
                axes[1, j].axis('off')

    plt.tight_layout()
    plt.show()

Kemudian, Tahap ini bertujuan untuk memeriksa integritas semua file gambar `.jpg` dalam folder. Setiap gambar dicek apakah bisa dibuka tanpa error—jika ya, dicetak sebagai "berhasil dibuka", jika tidak, ditandai sebagai rusak atau error. Proses ini penting untuk memastikan dataset bersih dan bebas dari file korup sebelum digunakan dalam analisis atau pelatihan model.

In [ ]:
for img_file in os.listdir(folder_path):
    if img_file.lower().endswith('.jpg'):
        img_path = os.path.join(folder_path, img_file)
        if os.path.isfile(img_path):
            try:
                img = Image.open(img_path)
                img.verify()
                print(f"{img_file} : Berhasil dibuka (tidak rusak)")
            except Exception as e:
                print(f"{img_file} : ERROR - Tidak bisa dibuka atau rusak ({e})")

# B. Split Data Train & Test

pada tahap split data ini, setiap gambar `.jpg` dalam `image_files` dibuka, diubah ke RGB, dan di-resize menjadi 100x100 piksel. Gambar asli disimpan ke list `images`, dan nama filenya ke `labels`. Jika opsi `add_noise` diaktifkan, versi bising dari gambar ditambahkan menggunakan noise Gaussian (dengan standar deviasi 15), lalu ikut dimasukkan ke dataset. Selanjutnya, seluruh data diubah ke array NumPy dan dibagi secara acak: 80% untuk pelatihan, sisanya dibagi rata untuk validasi dan pengujian. Proses ini memastikan bahwa model nantinya dilatih dengan data yang bersih dan juga data dengan variasi (noise), untuk meningkatkan kemampuan generalisasi.

Tahap ini bertujuan untuk mempersiapkan dataset gambar berukuran seragam dan memperbanyak data dengan menambahkan noise, lalu membaginya ke dalam subset pelatihan, validasi, dan pengujian.


In [ ]:
image_size = (100, 100)
add_noise = True

def add_gaussian_noise(image_array, mean=0, std=15):
    noise = np.random.normal(mean, std, image_array.shape)
    noisy_image = image_array + noise
    noisy_image = np.clip(noisy_image, 0, 255).astype(np.uint8)
    return noisy_image

images, labels = [], []

for file_path in image_files:
    try:
        with Image.open(file_path) as img:
            img = img.convert('RGB')
            img = img.resize(image_size)
            img_arr = np.array(img)
            images.append(img_arr)
            labels.append(os.path.basename(file_path))

            if add_noise:
                noisy_version = add_gaussian_noise(img_arr)
                images.append(noisy_version)
                labels.append(os.path.basename(file_path) + "_noise")
    except Exception as err:
        print(f"Gagal membaca file {file_path}: {err}")

images = np.array(images)
labels = np.array(labels)

X_train, X_temp, y_train, y_temp = train_test_split(images, labels, test_size=0.2, random_state=42, shuffle=True)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

print(f"Jumlah total data (termasuk noise): {len(images)}")
print(f"Data Train: {len(X_train)}, Validasi: {len(X_val)}, Test: {len(X_test)}")

Jumlah total data (termasuk noise): 2148
Data Train: 1718, Validasi: 215, Test: 215


# C. Baseline Model Autoencoder

Thap selanjutnya, yaitu membuat model baseline. Model diawali dengan layer input berukuran `(100, 100, 3)`, lalu bagian *encoder* terdiri dari beberapa layer `Conv2D` dan `MaxPooling2D` yang bertugas mengekstraksi fitur dan mengecilkan dimensi spasial gambar. Setelah mencapai representasi laten, bagian *decoder* menggunakan `UpSampling2D` dan `Conv2D` untuk memperbesar kembali dimensi ke ukuran semula. Layer terakhir memakai aktivasi `sigmoid` agar output berada pada rentang \[0, 1], cocok untuk tugas rekonstruksi gambar.

Model dikompilasi dengan optimizer Adam dan loss function *mean squared error* (MSE), yang umum digunakan untuk membandingkan kemiripan piksel antara input dan hasil rekonstruksi.

Tahap ini bertujuan untuk membangun dan mengompilasi model *autoencoder convolutional* yang digunakan untuk merekonstruksi gambar input, misalnya untuk denoising atau kompresi.


In [ ]:
input_img = layers.Input(shape=(100, 100, 3), name='image_input')

enc = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
enc = layers.MaxPooling2D((2, 2), padding='same')(enc)
enc = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(enc)
enc = layers.MaxPooling2D((2, 2), padding='same')(enc)
enc = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(enc)

dec = layers.UpSampling2D((2, 2))(enc)
dec = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(dec)
dec = layers.UpSampling2D((2, 2))(dec)
output_img = layers.Conv2D(3, (3, 3), activation='sigmoid', padding='same')(dec)

autoencoder = models.Model(inputs=input_img, outputs=output_img)

autoencoder.compile(optimizer='adam', loss='mse')
autoencoder.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ image_input (InputLayer)        │ (None, 100, 100, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 100, 100, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 50, 50, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 50, 50, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 25, 25, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 25, 25, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d (UpSampling2D)    │ (None, 50, 50, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 50, 50, 64)     │        73,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_1 (UpSampling2D)  │ (None, 100, 100, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 100, 100, 3)    │         1,731 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 168,771 (659.26 KB)

 Trainable params: 168,771 (659.26 KB)

 Non-trainable params: 0 (0.00 B)

# D. Modifikasi Model Autoencoder

Tahap ini membangun model autoencoder versi modifikasi dengan menambahkan *Batch Normalization* setelah setiap layer konvolusi untuk meningkatkan stabilitas dan efisiensi pelatihan. Arsitektur terdiri dari encoder yang mengecilkan dimensi gambar dan decoder yang merekonstruksinya kembali ke ukuran semula. Model dikompilasi dengan optimizer Adam dan loss MSE untuk meminimalkan perbedaan antara input dan output gambar.

In [ ]:
input_img = layers.Input(shape=(100, 100, 3), name='image_input')

e = layers.Conv2D(32, (3, 3), padding='same', activation='relu')(input_img)
e = layers.BatchNormalization()(e)
e = layers.MaxPooling2D((2, 2), padding='same')(e)

e = layers.Conv2D(64, (3, 3), padding='same', activation='relu')(e)
e = layers.BatchNormalization()(e)
e = layers.MaxPooling2D((2, 2), padding='same')(e)

e = layers.Conv2D(128, (3, 3), padding='same', activation='relu')(e)
e = layers.BatchNormalization()(e)

d = layers.Conv2D(64, (3, 3), padding='same', activation='relu')(e)
d = layers.BatchNormalization()(d)
d = layers.UpSampling2D((2, 2))(d)

d = layers.Conv2D(32, (3, 3), padding='same', activation='relu')(d)
d = layers.BatchNormalization()(d)
d = layers.UpSampling2D((2, 2))(d)

output_img = layers.Conv2D(3, (3, 3), padding='same', activation='sigmoid')(d)

autoencoder_mod = models.Model(inputs=input_img, outputs=output_img)
autoencoder_mod.compile(optimizer='adam', loss='mse')
autoencoder_mod.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ image_input (InputLayer)        │ (None, 100, 100, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 100, 100, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 100, 100, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 50, 50, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 50, 50, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 50, 50, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 25, 25, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 25, 25, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 25, 25, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 25, 25, 64)     │        73,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 25, 25, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_2 (UpSampling2D)  │ (None, 50, 50, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 50, 50, 32)     │        18,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 50, 50, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_3 (UpSampling2D)  │ (None, 100, 100, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 100, 100, 3)    │           867 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 187,651 (733.01 KB)

 Trainable params: 187,011 (730.51 KB)

 Non-trainable params: 640 (2.50 KB)

# E. Evaluasi Model

Terakhir dilakukannya evaluasi, dengan fungsi `compute_avg_ssim` menghitung rata-rata SSIM antara gambar asli dan hasil rekonstruksi. SSIM mengukur kesamaan struktur visual, sehingga semakin mendekati 1 nilainya, semakin baik hasil rekonstruksinya. Nilai SSIM dari kedua model ditampilkan sebagai pembanding performa visual.

Tahap ini bertujuan untuk mengevaluasi kualitas rekonstruksi gambar oleh model autoencoder dengan menggunakan metrik SSIM. Data uji `X_test` terlebih dahulu dinormalisasi ke rentang \[0, 1], lalu masing-masing model baseline dan modifikasi digunakan untuk merekonstruksi gambar.


In [ ]:
X_test_scaled = X_test.astype("float32") / 255.0

recon_base = autoencoder.predict(X_test_scaled)
recon_mod = autoencoder_mod.predict(X_test_scaled)

def compute_avg_ssim(original, reconstructed):
    ssim_scores = [
        ssim(orig, recon, channel_axis=-1, data_range=1.0)
        for orig, recon in zip(original, reconstructed)
    ]
    return np.mean(ssim_scores)

avg_ssim_base = compute_avg_ssim(X_test_scaled, recon_base)
avg_ssim_mod = compute_avg_ssim(X_test_scaled, recon_mod)

print(f"Average SSIM Baseline   : {avg_ssim_base:.4f}")
print(f"Average SSIM Modifikasi : {avg_ssim_mod:.4f}")

7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 380ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 272ms/step
Average SSIM Baseline   : 0.0671
Average SSIM Modifikasi : 0.0702


Kesimpulannya, implementasi dan evaluasi dua arsitektur autoencoder menunjukkan bahwa penambahan *Batch Normalization* pada model modifikasi memberikan dampak positif terhadap kualitas rekonstruksi gambar. Hal ini dibuktikan dari peningkatan nilai rata-rata SSIM, di mana model modifikasi mencapai **0.0702**, sedikit lebih baik dibandingkan model baseline dengan nilai **0.0671**. Meskipun selisihnya tipis, hasil ini mengindikasikan bahwa normalisasi membantu model belajar representasi fitur yang lebih stabil dan akurat, sehingga mampu menghasilkan gambar hasil rekonstruksi yang lebih menyerupai aslinya.


# F. Video Penjelasan

Video terdapat pada link berikut:

https://drive.google.com/drive/folders/1i66HebEyFdobM4UYxXD7Z4uCOQKslo8L?usp=drive_link